In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [42]:
import os
import re
import time
from typing import List, Any, Union
import pandas as pd
import multiprocessing as mp
import copy as cp
import math

In [54]:

import os
def buildGraph(data):
    graph = {}
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        graph[line[0]] = line[1:]
    return graph

def reFormatTestData(data):
    format_data = []
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        line[0] = int(line[0])
        format_data.append(line)
    return format_data

# read train data
file_dir = 'comp90051-2020-sem2-proj1'
file_name = "SmallTrain"
with open(os.path.join(file_dir, file_name)) as f:
    graph = f.readlines()
    graph = buildGraph(graph)

# read test data
test_name = "SmallTest"
with open(os.path.join(file_dir, test_name)) as f:
    f.readline()
    test_data = f.readlines()
    test_data = reFormatTestData(test_data)

print(len(graph))
print(len(test_data))

20000
2


In [55]:
'''
### Edge Set Building
'''
def buildEdgeSet(graph):
    edge_set = set()
    # Positive train data
    for key in graph:
        for each in graph[key]:
            edge_set.add((key,each))
    return edge_set

edge_set = buildEdgeSet(graph)

print(len(edge_set))

23946602


In [56]:
# read train data and label
file_dir = '../input/trainandvalidationdata'
file_name = "train_data.csv"
with open(os.path.join(file_dir, file_name)) as f:
    f.readline()
    train_data = []
    data = f.readlines()
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(',', line)
        line[0] = int(line[0])
        train_data.append(line)

file_name = "train_label.csv"
with open(os.path.join(file_dir, file_name)) as f:
    f.readline()
    train_label = f.readlines()

# read validation data and label
file_name = "validation_data.csv"
with open(os.path.join(file_dir, file_name)) as f:
    f.readline()
    validation_data = []
    data = f.readlines()
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(',', line)
        line[0] = int(line[0])
        validation_data.append(line)

file_name = "validation_label.csv"
with open(os.path.join(file_dir, file_name)) as f:
    f.readline()
    validation_label = f.readlines()

print(len(train_data))
print(train_data[0])
print(len(train_label))
print(train_label[0])

print(len(validation_data))
print(validation_data[0])
print(len(validation_label))
print(validation_label[0])

80000
[0, '2722942', '243804']
80000
0,1

20000
[0, '1063358', '421585']
20000
0,1



In [57]:
'''
### Feature Extraction
'''

'''
##顶点度特征
'''
# feature 1: out,关注的人
def getDegreeOut(node, graph):
    degreeOut = set()
    try:
        degreeOut = set(graph[node])
        return degreeOut
    except:
        return degreeOut

# feature 2: in 关注我的人
def getDegreeIn(node, graph):
    degreeIn = set()
    for key, value in graph.items():
        if node in set(value):
            degreeIn.add(key)
    return degreeIn

# feature3: Combin in and out
def getNeighbors(node, graph):
    degree_out = getDegreeOut(node, graph)
    degree_in = getDegreeIn(node, graph)
    neighbors = degree_out.union(degree_in)
    return neighbors

# feature 4: The Intersection of out and in 就是互关
def feature4(a, graph):
    out_a = getDegreeOut(a, graph)
    in_a = getDegreeIn(a, graph)
    return len(out_a.intersection(in_a))


'''
##共同邻居特征
'''
# feature 5: Common Neighbours of A and B
def commonNeighbors(a, b, graph):
    neighbors_a = getNeighbors(a, graph)
    neighbors_b = getNeighbors(b, graph)
    return len(neighbors_a.intersection(neighbors_b))

# feature 6: Common Followees(in) of A and B
def commonIn(a, b, graph):
    in_a = getDegreeIn(a, graph)
    in_b = getDegreeIn(b, graph)
    return len(in_a.intersection(in_b))

# feature 7: Common Followers(out) of A and B
def commonOut(a, b, graph):
    out_a = getDegreeOut(a, graph)
    out_b = getDegreeOut(b, graph)
    return len(out_a.intersection(out_b))

'''
##邻居总数特征
'''
# feature 8: Union Followers(out) of A and B
def unionOut(a, b, graph):
    out_a = getDegreeOut(a, graph)
    out_b = getDegreeOut(b, graph)
    return len(out_a.union(out_b))

# feature 9: Union Followees(in) of A and B
def unionIn(a, b, graph):
    in_a = getDegreeIn(a, graph)
    in_b = getDegreeIn(b, graph)
    return len(in_a.union(in_b))

# feature 10: Union Neighbours(out) of A and B
def unionNeighbours(a, b, graph):
    neighbours_a = getNeighbors(a, graph)
    neighbours_b = getNeighbors(b, graph)
    return len(neighbours_a.union(neighbours_b))

'''
##中介邻居特征
'''
# feature 11: Union Out of A and In of B
def unionAoutBin(a, b, graph):
    out_a = getDegreeOut(a, graph)
    in_b = getDegreeIn(b, graph)
    return len(out_a.union(in_b))

'''
##优先链接特征
'''
# feature 12: Product Neighbours of A and Neighbours of B
def productAneighboursBneighbours(a, b, graph):
    score = len(getNeighbors(a,graph)) * len(getNeighbors(b,graph))
    return score

'''
##邻居评价特征
'''
# feature 13: all sum of edge exist between all neighbours of A and all neighbours of B
def neighboursMeasure(a, b, graph):
    neighbours_a = getNeighbors(a,graph)
    neighbours_b = getNeighbors(b,graph)
    score = 0
    for each_a in neighbours_a:
        for each_b in neighbours_b:
            if(each_a == each_b or (each_a,each_b) in edge_set or (each_b,each_a) in edge_set):
                score+=1
    return score

'''
##反向关系特征
'''
# feature 14: all sum of edge exist between all neighbours of A and all neighbours of B
def oppsiteDirection(a, b, graph):
    score = 0
    if (b,a) in edge_set:
        score = 1
    return score


'''
##邻居子图特征
'''
# feature 15: Subgraph of neighbours 1
def neighboursSubgraph1(a, b, graph):
    neighbours_a = getNeighbors(a,graph)
    neighbours_b = getNeighbors(b,graph)
    neighbours = list(neighbours_a.intersection(neighbours_b))
    score = 0
    for each_a in neighbours:
        for each_b in neighbours:
            if((each_a,each_b) in edge_set):
                score+=1
    return score

# feature 16: Subgraph of neighbours 2
def neighboursSubgraph2(a, b, graph):
    neighbours_a = getNeighbors(a,graph)
    neighbours_b = getNeighbors(b,graph)
    neighbours = neighbours_a.intersection(neighbours_b)
    neighbours.add(a)
    neighbours.add(b)
    neighbours = list(neighbours)
    score = 0
    for each_a in neighbours:
        for each_b in neighbours:
            if((each_a,each_b) in edge_set):
                score+=1
    return score


# feature 17
def GetJaccardCoefficient(user_X,user_y,graph):
    neighbors_x = getNeighbors(user_X, graph)
    neighbors_y = getNeighbors(user_y, graph)
    CommonNeighbors=len(neighbors_x.intersection(neighbors_y))
    AllNeighborss=len(neighbors_x.union(neighbors_y))
    if AllNeighborss>=0:
        JaccardCoefficient=CommonNeighbors/AllNeighborss
    return JaccardCoefficient

#feature 18 Adamic
#邻居的邻居数量越少，就越凸显它作为“中间人”的重要性，毕竟一共只认识那么少人，却恰好是x，y的好朋友

def GetAdamic(user_X,user_y,graph):
    neighbors_x = getNeighbors(user_X, graph)
    neighbors_y = getNeighbors(user_y, graph)
    CommonNeighbors=neighbors_x.intersection(neighbors_y)
    Adamic=0
    for commonNeighbor in CommonNeighbors:
        NeighborOfCommonNeighbor=len(getNeighbors(commonNeighbor,graph))
        if NeighborOfCommonNeighbor!=0 and NeighborOfCommonNeighbor!=1:
            Adamic+=1/math.log(NeighborOfCommonNeighbor)

    return Adamic

#feature 19
#sumCoauthor
def GetSumConeighbor(user_X,user_y,graph):
    print("19")
    neighbors_x = getNeighbors(user_X, graph)
    neighbors_y = getNeighbors(user_y, graph)
    SumCoNeighbor=0
    for Neighbor in neighbors_x:
        print("a")
        NeighborOfNeighbor=len(getNeighbors(Neighbor,graph))
        if NeighborOfNeighbor==1:
            SumCoNeighbor+=1
    for Neighbor in neighbors_y:
        print("b")
        NeighborOfNeighbor=len(getNeighbors(Neighbor,graph))
        if NeighborOfNeighbor==1:
            SumCoNeighbor+=1
    print("20")
    return SumCoNeighbor

debugging methods

In [58]:
# Feature Extraction
def getFeatures(record: list) -> list:
    node = [record[0],
            productAneighboursBneighbours(record[1],record[-1],graph),
            neighboursMeasure(record[1],record[-1],graph),
            oppsiteDirection(record[1],record[-1],graph),
            neighboursSubgraph1(record[1],record[-1],graph),
            neighboursSubgraph2(record[1],record[-1],graph),
            GetJaccardCoefficient(record[1],record[-1],graph),
            GetAdamic(record[1],record[-1],graph)]
    return node


In [59]:
print(len(validation_data))
validation_data1 = validation_data[0:1000] + validation_data[10000:11000]
print(len(validation_data1))


20000
20


In [60]:

# Feature Extraction for validation data
validation_data1 = validation_data[0:1000] + validation_data[10000:11000]

features=[]
for record in validation_data1:
    features.append(getFeatures(record))

title = ["Id", "CommonNeighboursLevel1"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('test_set_features_test.csv', encoding='utf-8')


20


KeyboardInterrupt: 

following can be ignored.


In [49]:
'''
### Train Set Building
'''
def buildPositiveTrainSet(temp_graph):
    train_set_positive = set()
    # Positive train data
    for key in temp_graph:
        for each in graph[key]:
            train_set_positive.add((key,each,1)) #setting category of positive as 1 eg:(seed,followee1,1)
    return list(train_set_positive)

train_set_positive = buildPositiveTrainSet(graph)
print(train_set_positive)
print(len(train_set_positive))

In [49]:
# Feature Extraction
def getTrainFeatures(record):
    node= [record[0],len(GetFans(record[1],graph)),len(GetFollows(record[1],graph)),len(GetFans(record[-1],graph)),len(GetFollows(record[-1],graph)),commonNeighborsLevel1(record[1], record[-1], graph),record[-1]]
    return node

print(train_set)




title = ["Id", "InSource", "OutSource", "InSink", "OutSink", "CommonNeighboursLevel1"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('./train_set_positive_features.csv', encoding='utf-8')

features.clear()
test_pd.clear()

In [49]:
def buildNegativeTrainSet(graph):
    train_set_negative = set()
    # Positive train data
    train_set_negative = set()
    for key in graph:
        for each in graph[key]:
            if((each,key) not in train_set_positive):
                train_set_negative.add((each,key,0))

    keys = list(graph.keys())
    for i in range(len(keys)-1):
        if ((keys[i],keys[i+1]) not in train_set_positive):
            train_set_negative.add((keys[i],keys[i+1]))
        if ((keys[i+1],keys[i]) not in train_set_positive):
            train_set_negative.add((keys[i+1],keys[i]))
    return list(train_set_negative)
#print(len(train_set_negative))
train_set_positive.clear()

In [49]:
# Feature Extraction
def getTrainFeatures(record):
    node= [record[0],len(GetFans(record[1],graph)),len(GetFollows(record[1],graph)),len(GetFans(record[-1],graph)),len(GetFollows(record[-1],graph)),commonNeighborsLevel1(record[1], record[-1], graph),record[-1]]
    return node

pool = mp.Pool(mp.cpu_count())
features = pool.map(getTrainFeatures, train_set)
pool.close()
pool.join()

title = ["Id", "InA", "OutA", "InB", "OutB", "CommonNeighboursLevel1", "Label"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('./train_set_negatitive_features.csv', encoding='utf-8')